In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [32]:
import requests
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import pandas as pd
import time

In [37]:
driver = webdriver.Chrome()
# Менять ссылку на нужный фильтр
url = 'https://www.booking.com/searchresults.ru.html?label=gen173nr-1BCAEoggI46AdIM1gEaIABiAEBmAEhuAEHyAEN2AEB6AEBiAIBqAIDuAKX8p64BsACAdICJDgyYjk4NTBiLTI3OTUtNDBkOS05YTRiLWQyNmNlYWFhZGE4ZtgCBeACAQ&sid=5b1e68297757b2d65bb54540773242bc&aid=304142&ss=Нур-Султан&ssne=Нур-Султан&ssne_untouched=Нур-Султан&lang=ru&src=searchresults&dest_id=-2334218&dest_type=city&checkin=2024-11-30&checkout=2024-12-01&group_adults=1&no_rooms=1&group_children=0&nflt=price%3DKZT-min-max-1%3Bht_id%3D204'
titles = []
prices = []
reviews = []
distances = []
types_of_room = []
accomodations = []
cities = []
taxes = []


driver.get(url)
time.sleep(100)

button = driver.find_element(By.XPATH, '//button[@aria-label="Скрыть меню входа в аккаунт."]')
button.click()
time.sleep(20)

previous_height = driver.execute_script('return document.body.scrollHeight')
while True:
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(40)
    new_height = driver.execute_script('return document.body.scrollHeight')
    if new_height == previous_height:
        break
    previous_height = new_height

while True:
    try:
        button = driver.find_element(By.XPATH, '//button[span[text()="Загрузить больше результатов"]]')
        button.click()
        time.sleep(40)
        
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(40)
        new_height = driver.execute_script('return document.body.scrollHeight')
        if new_height == previous_height:
            break
        previous_height = new_height
    except NoSuchElementException:
        break

page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')
blocks = soup.find_all('div', attrs={'data-testid': 'property-card'})

for block in blocks:
    class_name = 'title'
    class_review = 'review-score'
    score = 'ac4a7896c7'
    class_price = 'price-and-discounted-price'
    class_distance = 'distance'
    class_type_room = 'price-for-x-nights'
    class_accommodation = 'abf093bdfe e8f7c070a7'
    class_city = 'aee5343fdb def9bc142a'
    class_tax = 'taxes-and-charges'
    
    name = block.find('div', attrs={'data-testid': class_name}).text
    price = block.find('span', attrs={'data-testid': class_price}).text
    review_block = block.find('div', attrs={'data-testid': 'review-score'})
    review = 'not found'
    if type(review_block) == type(block):
        review = review_block.find('div', attrs={'class': score}).text
    distance = block.find('span', attrs={'data-testid': class_distance}).text
    type_of_room = block.find('div', attrs={'data-testid': class_type_room}).text
    accomodation = block.find('h4', attrs={'class': class_accommodation}).text
    city = block.find('span', attrs={'class': class_city}).text
    tax = block.find('div', attrs={'data-testid': class_tax}).text

    
    
    titles.append(name)
    prices.append(price)
    reviews.append(review)
    distances.append(distance)
    types_of_room.append(type_of_room)
    accomodations.append(accomodation)
    cities.append(city)
    taxes.append(tax)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@aria-label="Скрыть меню входа в аккаунт."]"}
  (Session info: chrome=130.0.6723.117); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104dc15dc cxxbridge1$str$ptr + 3653648
1   chromedriver                        0x0000000104db9e3c cxxbridge1$str$ptr + 3623024
2   chromedriver                        0x0000000104824100 cxxbridge1$string$len + 88404
3   chromedriver                        0x000000010486641c cxxbridge1$string$len + 359536
4   chromedriver                        0x000000010489fba8 cxxbridge1$string$len + 594940
5   chromedriver                        0x000000010485b0fc cxxbridge1$string$len + 313680
6   chromedriver                        0x000000010485bd4c cxxbridge1$string$len + 316832
7   chromedriver

In [34]:
print(len(titles))

df = pd.DataFrame({
    'name': titles,
    'price': prices,
    'review': reviews,
    'distance': distances,
    'type of room': types_of_room,
    'accomodation condition': accomodations,
    'city': cities,
    'taxes': taxes,
    'guests': 3,
    'kids': 1
})

df


0


,name,price,review,distance,type of room,accomodation condition,city,taxes,guests,kids


In [6]:
# df.to_csv('Almaty_1_0.csv', index=False, encoding='utf-8', sep=',', header=True) # Поменять имя файла если нужно

In [9]:
# df.to_csv('Almaty_2_1.csv', index=False, encoding='utf-8', sep=',', header=True)

In [14]:
# df.to_csv('Almaty_2_0.csv', index=False, encoding='utf-8', sep=',', header=True)

In [17]:
# df.to_csv('Almaty_2_2.csv', index=False, encoding='utf-8', sep=',', header=True)

In [29]:
df.to_csv('Astana_1_0.csv', index=False, encoding='utf-8', sep=',', header=True)